In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sys
sys.path.append("..")
import scipy.constants as const
import energy_utils as energy
import matplotlib.cm as cm
# Enable or disable Tensor Float 32 Execution
tf.config.experimental.enable_tensor_float_32_execution(False)

import matplotlib.pyplot as plt
from scipy.integrate import simpson

params = {"axes.labelsize": 14,
          "axes.titlesize": 16,}
plt.rcParams["axes.linewidth"] = 1
plt.rcParams['mathtext.bf'] = 'STIXGeneral:italic:bold'
plt.rcParams['figure.dpi'] = 100
plt.rcParams.update(params)

def place(ax):
  ax.tick_params(direction="in", which="minor", length=3)
  ax.tick_params(direction="in", which="major", length=5, labelsize=13)
  ax.grid(which="major", ls="dashed", dashes=(1, 3), lw=0.8, zorder=0)
  #ax.legend(frameon=True, loc="best", fontsize=12,edgecolor="black")
  fig.tight_layout()

In [ ]:

popt_loaded = np.load("../../data/dielectric_function_fitted.npy")

def dielectric_function(rho, T, a0, a1, a2, a3,
                        b1, b2, b3, b4, b5, 
                        c1, c2, c3, c4, c5):
    """
    function for dielectric constant ε(ρ, T) with a 5th-order polynomial in ρ
    """
    return np.exp(a0 + a1*T + a2*T**2 + a3*T**3 
                  + b1*rho + b2*rho**2 + b3*rho**3 + b4*rho**4 + b5*rho**5 
                  + c1*T*rho + c2*T*rho**2 + c3*T*rho**3 + c4*T*rho**4 + c5*T*rho**5)


In [ ]:
model_path = "../../models/c1_dipole_Mar9.keras"
model_c1 = keras.models.load_model(model_path)
dx = 0.02

In [ ]:


def beta_mu_correction(sigma, T, epsilon, rho, dipole=0.382):
    prefactor = const.elementary_charge**2 /(4 * const.pi * const.epsilon_0 * 1e-10 )
    
    beta = 1/ (const.Boltzmann * T) 
    
    first = -  2 * dipole**2 / ( 3 * sigma**3 * np.pi**0.5  )
   
    second = 2*(epsilon-1) / (epsilon*4*np.pi * rho * np.pi**0.5 * sigma**3)  
  
    return prefactor*first*beta + second 



def get_betamu(T, E_field):
    rho_range = np.linspace(0.00001, 0.035, 80)

    betamu_SR = np.empty_like(rho_range)
    betamu_LR = np.empty_like(rho_range)
    z_range = np.ones(1)
    elec_array = np.zeros(1) + E_field / (const.Boltzmann * T / const.elementary_charge)

    for i in range(len(rho_range)):
        
        rho_array = z_range * rho_range[i]
        betamuSR = energy.get_betamu(T, rho_array, elec_array, model_c1, dx)
        epsilon =  np.exp(dielectric_function(rho_range[i], T, *popt_loaded))
        if rho_range[i] < 0.001:
            betamu_correction = 0.0
        else:
            betamu_correction = beta_mu_correction(4.5, T, epsilon, rho_range[i])
        
        betamu_SR[i] = betamuSR
        betamu_LR[i] = betamuSR + betamu_correction
        
        
        
    
    return rho_range, betamu_SR, betamu_LR

In [ ]:


fig, ax = plt.subplots(1, 1, figsize=(4,4), sharex=True)


temp_range = np.arange(250, 550, 50)
colors = cm.RdPu(np.linspace(0, 1, len(temp_range)+2))
for i, temp in enumerate(temp_range):

    rho, betamu_SR, betamu_LR = get_betamu(temp, 0.0)

    ax.plot(rho, betamu_SR, lw=1, ls='--', label=f"{temp}, SR", color=colors[i+2])
    ax.plot(rho, betamu_LR, lw=1, label=f"{temp}, LR", color=colors[i+2])


ax.set_xlabel(r"$\rho$ $[\mathrm{\AA}^{-3}]$")
ax.set_ylabel(r"$\beta\mu$")
ax.legend(title=r"$T$ [K]", fontsize=9,ncols=2)
ax.set_xlim(0, 0.032)
place(ax)


In [ ]:
from scipy.signal import argrelextrema
from scipy.integrate import simpson
from scipy.interpolate import interp1d

def spinodal_idx(chemical_potential):
    # Return the indices corresponding to the max and min mu in the vdw loop

    # Find the local minima and maxima (for the Van der Waals loop)
    minima = argrelextrema(chemical_potential, np.less)[0]
    maxima = argrelextrema(chemical_potential, np.greater_equal)[0]

    # Extract the first minimum and maximum to form the Van der Waals loop
    loop_min_idx = minima[0]
    loop_max_idx = maxima[0]

    return loop_max_idx, loop_min_idx

def roots(density, chemical_potential, target_mu, tol=9e-4, max_iter=10000):
    # Newton-Raphson method to calculate where the coexistence mu crosses the equation of state
    # to find rho_liq and rho_gas

    # Interpolation of mu vs density
    mu_interp = interp1d(density, chemical_potential, kind="cubic", fill_value="extrapolate")

    # Derivative of the interpolated mu with respect to density
    mu_deriv_interp = interp1d(density, np.gradient(chemical_potential, density), kind="cubic", fill_value="extrapolate")

    # Function for finding roots
    def f(rho):
        return mu_interp(rho) - target_mu

    # Derivative of the function
    def df(rho):
        return mu_deriv_interp(rho)

    # Initial guesses: Take initial points near where the sign change occurs
    intersection_indices = np.where(np.diff(np.sign(mu_interp(density) - target_mu)))[0]
    
    if len(intersection_indices) < 3:
        return np.array([]), np.array([])  # No roots found

    rho_intersections = []
    mu_intersections = []

    # Try multiple starting points based on where sign change occurs
    for idx in intersection_indices:
        initial_guess = density[idx]
        rho = initial_guess

        for _ in range(max_iter):
            # Newton-Raphson update
            f_value = f(rho)
            df_value = df(rho)

            if np.abs(df_value) < 1e-10:  # Avoid division by zero
                break

            # Update the guess
            rho_new = rho - f_value / df_value

            if np.abs(rho_new - rho) < tol:  # Convergence check
                rho_intersections.append(rho_new)
                mu_intersections.append(mu_interp(rho_new))
                break

            # Update rho for the next iteration
            rho = rho_new

    return np.array(rho_intersections)


def calc_area(density, chemical_potential, density_roots, target_mu):
    density_roots.sort()

    # Interpolate the chemical potential as a continuous function of density
    interpolation = interp1d(density, chemical_potential, kind='cubic', fill_value='extrapolate')

    # Define the density ranges based on roots
    first_root, second_root, third_root = density_roots

    # Generate a finer density grid to perform integration
    fine_density1 = np.linspace(first_root, second_root, 100)
    fine_density2 = np.linspace(second_root, third_root, 100)

    # Interpolate chemical potential over the finer density grid
    interpolated_mu1 = interpolation(fine_density1)
    interpolated_mu2 = interpolation(fine_density2)

    # Calculate the area difference using Simpson's rule
    A_1 = simpson(interpolated_mu1 - target_mu, x=fine_density1)
    A_2 = simpson(target_mu - interpolated_mu2, x=fine_density2)

    deltaA = A_1 - A_2

    return deltaA, fine_density1, fine_density2, interpolated_mu1, interpolated_mu2


def calc_mu_coex(density, chemical_potential, tol=1e-4, iter=100000):
    # tol is tolerance in area

    idx1, idx2 = spinodal_idx(chemical_potential)

    mu_guess = 0.5*(chemical_potential[idx1] + chemical_potential[idx2])
    # mean of local min and max

    for i in range(iter):
        #print(density)
        #print(chemical_potential)
        #print(mu_guess)
        delta = calc_area(density, chemical_potential, roots(density, chemical_potential, mu_guess), mu_guess)[0]

        if np.abs(delta) < tol:
            break

        elif i == iter-1:
            print("Maxwell mu has not converged; delta = {}".format(delta))

        elif delta > 0:
            mu_guess += 1e-5

        elif delta < 0:
            mu_guess -= 1e-5

    #print(f"mu_coex = {mu_guess}; area difference = {delta}")

    return mu_guess


def extract_rho_coex(density, chemical_potential):
    mu_coex = calc_mu_coex(density, chemical_potential)
  
    root = roots(density, chemical_potential, mu_coex)
    root.sort()

    areas = calc_area(density, chemical_potential, root, mu_coex)[1:]

    return root, mu_coex, areas

 

In [ ]:

range_temp = np.arange(250, 401, 5)
rho_v_LR = np.zeros(len(range_temp))
rho_m_LR = np.zeros(len(range_temp))
rho_l_LR = np.zeros(len(range_temp))
rho_v_SR = np.zeros(len(range_temp))
rho_m_SR = np.zeros(len(range_temp))
rho_l_SR = np.zeros(len(range_temp))
mucoex_LR = np.zeros(len(range_temp))
mucoex_SR = np.zeros(len(range_temp))

for i, temp in enumerate(range_temp):
    print("Calculating for T = {}".format(temp))    
    rho, betamu_SR, betamu_LR  = get_betamu(temp, 0.0)
    roots_SR, mu_coex_SR, areas_SR  = extract_rho_coex(rho, betamu_SR)
    roots_LR, mu_coex_LR, areas_LR  = extract_rho_coex(rho, betamu_LR)
    rho_v_SR[i] = roots_SR[0]
    rho_m_SR[i] = roots_SR[1]
    rho_l_SR[i] = roots_SR[2]
    mucoex_SR[i] = mu_coex_SR
    rho_v_LR[i] = roots_LR[0]
    rho_m_LR[i] = roots_LR[1]
    rho_l_LR[i] = roots_LR[2]
    mucoex_LR[i] = mu_coex_LR
    

In [ ]:



fig, ax = plt.subplots(1, 1, figsize=(4,4), sharex=True)


ax.plot(rho_v_SR, range_temp, lw=1, color="red", label=r"SR")
#ax.plot(rho_m_SR, range_temp, lw=1, color="red", ls='--')
ax.plot(rho_l_SR, range_temp, lw=1, color="red")


ax.plot(rho_v_LR, range_temp, lw=1, color="blue", label=r"LR")
#ax.plot(rho_m_LR, range_temp, lw=1, color="blue", ls='--')
ax.plot(rho_l_LR, range_temp, lw=1, color="blue")
ax.set_xlabel(r"$\rho$ $[\mathrm{\AA}^{-3}]$")
ax.set_xlabel(r"$T$ [K]")

ax.legend()
place(ax)